# Read Data Sample

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows",15)
%matplotlib inline

In [2]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")
    
    kdd_train_5labels = pd.read_pickle("dataset/kdd_train_5labels.pkl")
    kdd_test_5labels = pd.read_pickle("dataset/kdd_test_5labels.pkl")
    

In [3]:
dataset.kdd_train_2labels.shape

(125973, 124)

In [4]:
dataset.kdd_test_2labels.shape

(22544, 124)

In [5]:
class preprocess:
    
    output_columns_2labels = ['is_Attack','is_Normal']

    from sklearn import model_selection as ms
    from sklearn import preprocessing as pp

    x_input = dataset.kdd_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.kdd_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.kdd_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.kdd_test_2labels.loc[:,output_columns_2labels]


    ss = pp.StandardScaler()

    x_train = ss.fit_transform(x_input)
    x_test = ss.transform(x_test_input)

    y_train = y_output.values

    x_valid, x_test, y_valid, y_test = ms.train_test_split(x_test, 
                                  y_test, 
                                  test_size=0.4)


In [6]:
import tensorflow as tf


In [16]:
class network(object):
    
    input_dim = 122
    classes = 2
    hidden_encoder_dim = 40
    hidden_layers = 1
    latent_dim = 10

    hidden_decoder_dim = 40
    
    def __init__(self, classes, hidden_layers, num_of_features):
        self.classes = classes
        self.hidden_layers = hidden_layers
        self.latent_dim = num_of_features
            
    def build_layers_vae(self):
        tf.reset_default_graph()
        #learning_rate = tf.Variable(initial_value=0.001)

        with tf.variable_scope("Input"):
            self.x = tf.placeholder("float", shape=[None, self.input_dim])
            self.y_ = tf.placeholder("float", shape=[None, self.classes])
            self.keep_prob = tf.placeholder("float")
        
        with tf.variable_scope("Layer_Encoder"):

            hidden_encoder = tf.layers.dense(self.x, self.hidden_encoder_dim, activation = tf.nn.relu)
            for h in range(self.hidden_layers - 1):
                hidden_encoder = tf.layers.dense(hidden_encoder, self.hidden_encoder_dim, activation = tf.nn.relu)
            
        with tf.variable_scope("Layer_Mean"):
            mu_encoder = tf.layers.dense(hidden_encoder, self.latent_dim, activation = None)

        with tf.variable_scope("Layer_Variance"):
            logvar_encoder = tf.layers.dense(hidden_encoder, self.latent_dim, activation = None)

        with tf.variable_scope("Sampling_Distribution"):
            # Sample epsilon
            epsilon = tf.random_normal(tf.shape(logvar_encoder), name='epsilon')

            # Sample latent variable
            std_encoder = tf.exp(0.5 * logvar_encoder)
            z = mu_encoder + tf.multiply(std_encoder, epsilon)
            tf.summary.histogram("Sample_Distribution", z)

        with tf.variable_scope("Layer_Decoder"):
            hidden_decoder = tf.layers.dense(z, self.hidden_decoder_dim, activation = tf.nn.relu)
            for h in range(self.hidden_layers - 1):
                hidden_decoder = tf.layers.dense(hidden_decoder, self.hidden_decoder_dim, activation = tf.nn.relu)

        with tf.variable_scope("Layer_Reconstruction"):
            x_hat = tf.layers.dense(hidden_decoder, self.input_dim, activation = tf.nn.relu)
        
        return z
    
    def build_layers_softmax(z):
        with tf.variable_scope("Layer_Dense_Hidden"):
            hidden_output = tf.layers.dense(z, self.latent_dim, activation=tf.nn.relu)

        with tf.variable_scope("Layer_Dense_Softmax"):
            y = tf.layers.dense(z, self.classes, activation=tf.nn.softmax)

        with tf.variable_scope("Loss"):
            BCE = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=x_hat, labels=self.x), reduction_indices=1)
            KLD = -0.5 * tf.reduce_mean(1 + logvar_encoder - tf.pow(mu_encoder, 2) - tf.exp(logvar_encoder), reduction_indices=1)
            softmax_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = self.y_, logits = y))

            loss = tf.reduce_mean(BCE + KLD + softmax_loss)

            self.regularized_loss = tf.abs(loss, name = "Regularized_loss")
            correct_prediction = tf.equal(tf.argmax(self.y_, 1), tf.argmax(y, 1))
            self.tf_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "Accuracy")

        with tf.variable_scope("Optimizer"):
            learning_rate=0.01
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.regularized_loss)
            
        # add op for merging summary
        self.summary_op = tf.summary.merge_all()
        self.pred = tf.argmax(y, 1)
        self.actual = tf.argmax(self.y_, 1)

        # add Saver ops
        # saver = tf.train.Saver()
        

In [18]:
class Train:    
    def train(epochs, net):
        batch_iterations = 100

        batch_indices = np.array_split(np.arange(preprocess.x_train.shape[0]), 
                                   batch_iterations)
        with tf.Session() as sess:
            summary_writer_train = tf.summary.FileWriter('./logs/kdd/VAE/training', graph=sess.graph)
            summary_writer_valid = tf.summary.FileWriter('./logs/kdd/VAE/validation')

            sess.run(tf.global_variables_initializer())

            for epoch in range(0, epochs):
                for i in batch_indices:
                    _, train_loss, summary_str = sess.run([net.optimizer, 
                                                           net.regularized_loss, 
                                                           net.summary_op],
                                                          feed_dict={net.x: preprocess.x_train[i,:], 
                                                                     net.y_: preprocess.y_train[i,:], 
                                                                     net.keep_prob:0.8})
                    summary_writer_train.add_summary(summary_str, epoch)


                accuracy, summary_str = sess.run([net.tf_accuracy, net.summary_op], 
                                                      feed_dict={net.x: preprocess.x_valid, 
                                                                 net.y_: preprocess.y_valid, 
                                                                 net.keep_prob:1})
                summary_writer_valid.add_summary(summary_str, epoch)

                if epoch % 10 == 0:
                    print("Step {} | Training Loss: {:.4f} | Validation Accuracy: {:.4f}".format(epoch, train_loss, accuracy))

            accuracy, pred_value, actual_value = sess.run([net.tf_accuracy, 
                                                           net.pred, 
                                                           net.actual], 
                                                          feed_dict={net.x: preprocess.x_test, 
                                                                     net.y_: preprocess.y_test, 
                                                                     net.keep_prob:1})


            print("Accuracy on Test data: {}".format(accuracy))
            return accuracy, pred_value, actual_value

In [19]:
import itertools
import collections
class Hyperparameters:
    #features_arr = [4, 8, 16, 32, 64, 128]
    #hidden_layers_arr = [2, 4, 6]
    features_arr = [8, 32, 64, 128, 256, 512]
    hidden_layers_arr = [2, 8]
    epochs = [30]
    result = collections.namedtuple("result", ["epochs", "hidden_layers", "feature_count",
                                  "accuracy"])
    results = []
    best_acc = 0
    for e, h, f in itertools.product(epochs, hidden_layers_arr, features_arr):
        print("Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
        n = network(2,h,f)
        z = n.build_layers_vae()
        n.build_layers_softmax(z)
        acc, pred, actual = Train.train(e, n)
        if acc > best_acc:
            best_acc = acc
            pred_value = pred
            actual_value = actual
        results.append(result(e, h, f,acc))

Current Layer Attributes - epochs:30 hidden layers:2 features count:8
[   0    1    2 ..., 1257 1258 1259]
[1260 1261 1262 ..., 2517 2518 2519]
[2520 2521 2522 ..., 3777 3778 3779]
[3780 3781 3782 ..., 5037 5038 5039]
[5040 5041 5042 ..., 6297 6298 6299]
[6300 6301 6302 ..., 7557 7558 7559]
[7560 7561 7562 ..., 8817 8818 8819]
[ 8820  8821  8822 ..., 10077 10078 10079]
[10080 10081 10082 ..., 11337 11338 11339]
[11340 11341 11342 ..., 12597 12598 12599]
[12600 12601 12602 ..., 13857 13858 13859]
[13860 13861 13862 ..., 15117 15118 15119]
[15120 15121 15122 ..., 16377 16378 16379]
[16380 16381 16382 ..., 17637 17638 17639]
[17640 17641 17642 ..., 18897 18898 18899]
[18900 18901 18902 ..., 20157 20158 20159]
[20160 20161 20162 ..., 21417 21418 21419]
[21420 21421 21422 ..., 22677 22678 22679]
[22680 22681 22682 ..., 23937 23938 23939]
[23940 23941 23942 ..., 25197 25198 25199]
[25200 25201 25202 ..., 26457 26458 26459]
[26460 26461 26462 ..., 27717 27718 27719]
[27720 27721 27722 ..., 28

[112124 112125 112126 ..., 113380 113381 113382]
[113383 113384 113385 ..., 114639 114640 114641]
[114642 114643 114644 ..., 115898 115899 115900]
[115901 115902 115903 ..., 117157 117158 117159]
[117160 117161 117162 ..., 118416 118417 118418]
[118419 118420 118421 ..., 119675 119676 119677]
[119678 119679 119680 ..., 120934 120935 120936]
[120937 120938 120939 ..., 122193 122194 122195]
[122196 122197 122198 ..., 123452 123453 123454]
[123455 123456 123457 ..., 124711 124712 124713]
[124714 124715 124716 ..., 125970 125971 125972]
[   0    1    2 ..., 1257 1258 1259]
[1260 1261 1262 ..., 2517 2518 2519]
[2520 2521 2522 ..., 3777 3778 3779]
[3780 3781 3782 ..., 5037 5038 5039]
[5040 5041 5042 ..., 6297 6298 6299]
[6300 6301 6302 ..., 7557 7558 7559]
[7560 7561 7562 ..., 8817 8818 8819]
[ 8820  8821  8822 ..., 10077 10078 10079]
[10080 10081 10082 ..., 11337 11338 11339]
[11340 11341 11342 ..., 12597 12598 12599]
[12600 12601 12602 ..., 13857 13858 13859]
[13860 13861 13862 ..., 15117 

[103311 103312 103313 ..., 104567 104568 104569]
[104570 104571 104572 ..., 105826 105827 105828]
[105829 105830 105831 ..., 107085 107086 107087]
[107088 107089 107090 ..., 108344 108345 108346]
[108347 108348 108349 ..., 109603 109604 109605]
[109606 109607 109608 ..., 110862 110863 110864]
[110865 110866 110867 ..., 112121 112122 112123]
[112124 112125 112126 ..., 113380 113381 113382]
[113383 113384 113385 ..., 114639 114640 114641]
[114642 114643 114644 ..., 115898 115899 115900]
[115901 115902 115903 ..., 117157 117158 117159]
[117160 117161 117162 ..., 118416 118417 118418]
[118419 118420 118421 ..., 119675 119676 119677]
[119678 119679 119680 ..., 120934 120935 120936]
[120937 120938 120939 ..., 122193 122194 122195]
[122196 122197 122198 ..., 123452 123453 123454]
[123455 123456 123457 ..., 124711 124712 124713]
[124714 124715 124716 ..., 125970 125971 125972]
[   0    1    2 ..., 1257 1258 1259]
[1260 1261 1262 ..., 2517 2518 2519]
[2520 2521 2522 ..., 3777 3778 3779]
[3780 3

[94498 94499 94500 ..., 95754 95755 95756]
[95757 95758 95759 ..., 97013 97014 97015]
[97016 97017 97018 ..., 98272 98273 98274]
[98275 98276 98277 ..., 99531 99532 99533]
[ 99534  99535  99536 ..., 100790 100791 100792]
[100793 100794 100795 ..., 102049 102050 102051]
[102052 102053 102054 ..., 103308 103309 103310]
[103311 103312 103313 ..., 104567 104568 104569]
[104570 104571 104572 ..., 105826 105827 105828]
[105829 105830 105831 ..., 107085 107086 107087]
[107088 107089 107090 ..., 108344 108345 108346]
[108347 108348 108349 ..., 109603 109604 109605]
[109606 109607 109608 ..., 110862 110863 110864]
[110865 110866 110867 ..., 112121 112122 112123]
[112124 112125 112126 ..., 113380 113381 113382]
[113383 113384 113385 ..., 114639 114640 114641]
[114642 114643 114644 ..., 115898 115899 115900]
[115901 115902 115903 ..., 117157 117158 117159]
[117160 117161 117162 ..., 118416 118417 118418]
[118419 118420 118421 ..., 119675 119676 119677]
[119678 119679 119680 ..., 120934 120935 120

[78120 78121 78122 ..., 79377 79378 79379]
[79380 79381 79382 ..., 80637 80638 80639]
[80640 80641 80642 ..., 81897 81898 81899]
[81900 81901 81902 ..., 83157 83158 83159]
[83160 83161 83162 ..., 84417 84418 84419]
[84420 84421 84422 ..., 85677 85678 85679]
[85680 85681 85682 ..., 86937 86938 86939]
[86940 86941 86942 ..., 88197 88198 88199]
[88200 88201 88202 ..., 89457 89458 89459]
[89460 89461 89462 ..., 90717 90718 90719]
[90720 90721 90722 ..., 91977 91978 91979]
[91980 91981 91982 ..., 93236 93237 93238]
[93239 93240 93241 ..., 94495 94496 94497]
[94498 94499 94500 ..., 95754 95755 95756]
[95757 95758 95759 ..., 97013 97014 97015]
[97016 97017 97018 ..., 98272 98273 98274]
[98275 98276 98277 ..., 99531 99532 99533]
[ 99534  99535  99536 ..., 100790 100791 100792]
[100793 100794 100795 ..., 102049 102050 102051]
[102052 102053 102054 ..., 103308 103309 103310]
[103311 103312 103313 ..., 104567 104568 104569]
[104570 104571 104572 ..., 105826 105827 105828]
[105829 105830 105831 ..

[73080 73081 73082 ..., 74337 74338 74339]
[74340 74341 74342 ..., 75597 75598 75599]
[75600 75601 75602 ..., 76857 76858 76859]
[76860 76861 76862 ..., 78117 78118 78119]
[78120 78121 78122 ..., 79377 79378 79379]
[79380 79381 79382 ..., 80637 80638 80639]
[80640 80641 80642 ..., 81897 81898 81899]
[81900 81901 81902 ..., 83157 83158 83159]
[83160 83161 83162 ..., 84417 84418 84419]
[84420 84421 84422 ..., 85677 85678 85679]
[85680 85681 85682 ..., 86937 86938 86939]
[86940 86941 86942 ..., 88197 88198 88199]
[88200 88201 88202 ..., 89457 89458 89459]
[89460 89461 89462 ..., 90717 90718 90719]
[90720 90721 90722 ..., 91977 91978 91979]
[91980 91981 91982 ..., 93236 93237 93238]
[93239 93240 93241 ..., 94495 94496 94497]
[94498 94499 94500 ..., 95754 95755 95756]
[95757 95758 95759 ..., 97013 97014 97015]
[97016 97017 97018 ..., 98272 98273 98274]
[98275 98276 98277 ..., 99531 99532 99533]
[ 99534  99535  99536 ..., 100790 100791 100792]
[100793 100794 100795 ..., 102049 102050 102051]

[57960 57961 57962 ..., 59217 59218 59219]
[59220 59221 59222 ..., 60477 60478 60479]
[60480 60481 60482 ..., 61737 61738 61739]
[61740 61741 61742 ..., 62997 62998 62999]
[63000 63001 63002 ..., 64257 64258 64259]
[64260 64261 64262 ..., 65517 65518 65519]
[65520 65521 65522 ..., 66777 66778 66779]
[66780 66781 66782 ..., 68037 68038 68039]
[68040 68041 68042 ..., 69297 69298 69299]
[69300 69301 69302 ..., 70557 70558 70559]
[70560 70561 70562 ..., 71817 71818 71819]
[71820 71821 71822 ..., 73077 73078 73079]
[73080 73081 73082 ..., 74337 74338 74339]
[74340 74341 74342 ..., 75597 75598 75599]
[75600 75601 75602 ..., 76857 76858 76859]
[76860 76861 76862 ..., 78117 78118 78119]
[78120 78121 78122 ..., 79377 79378 79379]
[79380 79381 79382 ..., 80637 80638 80639]
[80640 80641 80642 ..., 81897 81898 81899]
[81900 81901 81902 ..., 83157 83158 83159]
[83160 83161 83162 ..., 84417 84418 84419]
[84420 84421 84422 ..., 85677 85678 85679]
[85680 85681 85682 ..., 86937 86938 86939]
[86940 8694

[41580 41581 41582 ..., 42837 42838 42839]
[42840 42841 42842 ..., 44097 44098 44099]
[44100 44101 44102 ..., 45357 45358 45359]
[45360 45361 45362 ..., 46617 46618 46619]
[46620 46621 46622 ..., 47877 47878 47879]
[47880 47881 47882 ..., 49137 49138 49139]
[49140 49141 49142 ..., 50397 50398 50399]
[50400 50401 50402 ..., 51657 51658 51659]
[51660 51661 51662 ..., 52917 52918 52919]
[52920 52921 52922 ..., 54177 54178 54179]
[54180 54181 54182 ..., 55437 55438 55439]
[55440 55441 55442 ..., 56697 56698 56699]
[56700 56701 56702 ..., 57957 57958 57959]
[57960 57961 57962 ..., 59217 59218 59219]
[59220 59221 59222 ..., 60477 60478 60479]
[60480 60481 60482 ..., 61737 61738 61739]
[61740 61741 61742 ..., 62997 62998 62999]
[63000 63001 63002 ..., 64257 64258 64259]
[64260 64261 64262 ..., 65517 65518 65519]
[65520 65521 65522 ..., 66777 66778 66779]
[66780 66781 66782 ..., 68037 68038 68039]
[68040 68041 68042 ..., 69297 69298 69299]
[69300 69301 69302 ..., 70557 70558 70559]
[70560 7056

InvalidArgumentError: Nan in summary histogram for: Sampling_Distribution/Sample_Distribution
	 [[Node: Sampling_Distribution/Sample_Distribution = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Sampling_Distribution/Sample_Distribution/tag, Sampling_Distribution/add)]]

Caused by op 'Sampling_Distribution/Sample_Distribution', defined at:
  File "/home/ritesh_malaiya/anaconda3/envs/p3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ritesh_malaiya/anaconda3/envs/p3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ritesh_malaiya/anaconda3/envs/p3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ritesh_malaiya/anaconda3/envs/p3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ritesh_malaiya/anaconda3/envs/p3/lib/python3.6/site-packa

In [ ]:
df_results = pd.DataFrame(Hyperparameters.results)

In [ ]:
df_results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    np.set_printoptions(precision=4)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j].round(4),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import confusion_matrix
cm_2labels = confusion_matrix(y_pred = Hyperparameters.pred_value, y_true = Hyperparameters.actual_value)
plt.figure(figsize=[6,6])
plot_confusion_matrix(cm_2labels, preprocess.output_columns_2labels, normalize = True)